In [1]:
import torch
import numpy as np
import torch.nn as nn
from transformers import Blip2Processor, Blip2ForConditionalGeneration, AutoProcessor, Blip2ForImageTextRetrieval
from datasets import COCODataset
from tqdm import tqdm
from PIL import Image
from torch.utils.data import DataLoader, RandomSampler
# from utils import print_model_structure

from collections import defaultdict
from functools import partial

import gc
import inspect
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
model_name = "Salesforce/blip2-opt-2.7b"

model = Blip2ForConditionalGeneration.from_pretrained(model_name)
model = model.to(device)

processor = Blip2Processor.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/fs/nexus-scratch/vla/micromamba/envs/MMQ/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
model

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [5]:

coco_dataset = COCODataset(ann_file='/nfshomes/vla/project_dirs/low-bit-vision/datasets/cocow/annotations/captions_val2017.json',
                           img_dir='/nfshomes/vla/project_dirs/low-bit-vision/datasets/cocow/images/val2017')

# calibration_set = [coco_dataset[0], coco_dataset[1]]

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [6]:

len(coco_dataset)

5000

In [7]:
# utils

@torch.no_grad()
def compute_loss(fp_output, q_output):
    fp_output_flat = fp_output.view(-1)
    q_output_flat = q_output.view(-1)
    L2 = torch.linalg.norm(fp_output_flat - q_output_flat, ord=2)
    return L2


# returns all nn.linear within module (a layer)
def get_named_linears(module):
    return {name: mod for name, mod in module.named_modules() if isinstance(mod, nn.Linear)}


def sanitize_kwargs(inputs_kwargs, module):
    """
    Remove the arguments that are not supported in the module's
    forward pass

    Args:
        inputs_kwargs (`dict`):
            The input dictionary to pass to the model layer
        module (`torch.nn.Module`):
            Target module to quantize.
    """
    module_signature = inspect.signature(module.forward).parameters
    sanitized_kwargs = {}
    for k, v in inputs_kwargs.items():
        if k in module_signature:
            sanitized_kwargs[k] = v
    return sanitized_kwargs


In [8]:
# base class for AWQ quantizer
class BaseAWQQuantizer():
    
    def __init__(self, model, device, inputs_processor, dataset, **kwargs):
        self.model = model
        self.device = device
        self.inputs_processor = inputs_processor
        self.dataset = dataset

        # QUANTIZATION SETTINGS
        self.w_bits = 4
        self.group_size = 128
        self.grid_search_size = 20
        self.zero_point = True

        # TODO: change to something appropriate
        # AWQ uses 128 for LLMs
        self.n_samples = 2
        
        self.run_model = None


    
    def pseudo_quantize_tensor(self, w: torch.Tensor):
        org_w_shape = w.shape
        if self.group_size > 0:
            assert org_w_shape[-1] % self.group_size == 0
            w = w.reshape(-1, self.group_size)
        assert w.dim() == 2
        assert torch.isnan(w).sum() == 0

        # zero point quantization
        if self.zero_point:
            max_val = w.amax(dim=1, keepdim=True)
            min_val = w.amin(dim=1, keepdim=True)
            max_int = 2**self.w_bits - 1
            min_int = 0
            scales = (max_val - min_val).clamp(min=1e-5) / max_int
            zeros = (-torch.round(min_val / scales)).clamp_(min_int, max_int)
            w = (
                torch.clamp(torch.round(w / scales) + zeros, min_int, max_int) - zeros
            ) * scales
            zeros = zeros.view(org_w_shape[0], -1)
        else:
            max_val = w.abs().amax(dim=1, keepdim=True)
            max_val = max_val.clamp(min=1e-5)
            max_int = 2 ** (self.w_bits- 1) - 1
            min_int = -(2 ** (self.w_bits - 1))
            scales = max_val / max_int
            zeros = None
            w = torch.clamp(torch.round(w / scales), min_int, max_int) * scales

        assert torch.isnan(scales).sum() == 0
        assert torch.isnan(w).sum() == 0

        scales = scales.view(org_w_shape[0], -1)
        w = w.reshape(org_w_shape)

        return w, scales, zeros


    @torch.no_grad
    def quantize(self):

        layer_groups = self._get_model_layer_groups()
        calibration_set = self._get_calibration_set()
        first_inputs, self.layer_args, self.layer_kwargs = self._gather_first_inputs(layer_groups, calibration_set)


        for layer_group, modules in layer_groups.items():

            self.inps = first_inputs[layer_group]
            for i in tqdm(range(len(modules)), desc= f"Quantizing {layer_group}"):
                
                if layer_group != 'qformer_layers':

                    layer = modules[i]

                    # nn.linear modules within layer to quantize
                    named_linears = get_named_linears(layer)
                    linear_inputs = self._gather_linear_inputs(layer, named_linears, layer_group)

                    # for k,v in linear_inputs.items():
                    #     print(f'{k}:{v.shape}')

                    
                    grouped_mods = self._group_modules_for_scaling(layer, linear_inputs, layer_group)

                    scales = [
                        self._compute_scales(layer, **group)
                        for group in grouped_mods    
                    ]

                    # print(scales)
                    # print(torch.unique(scales[0]))
                    # print(torch.unique(scales[1]))
                    # print(torch.unique(scales[2]))

                    # apply scales to prev_op and modules
                    for group, scale in zip(grouped_mods, scales):
                        
                        print('-'*80)
                        print(group['prev_op'].weight)
                        print('-'*80)
                        self._apply_scales(scale, group['prev_op'], group['modules'])
                        print('-'*80)
                        print(group['prev_op'].weight)
                        print('-'*80)            


                    # TODO: solve for and apply clipping

                # TODO:remove
                break
            
            # TODO:remove
            # break

            
        return layer_groups, first_inputs, self.layer_args, self.layer_kwargs, linear_inputs, scales
  

    def _gather_first_inputs(self, layer_groups, calibration_set):

        first_inputs = {}
        layer_args = {}
        layer_kwargs = {}

        # get input and kwargs to layer 0
        # use this Catcher hack cause forward hooks cannot capture kwargs
        class Catcher(nn.Module):
            def __init__(self, module, layer_group, is_last):
                super().__init__()
                self.module = module
                self.layer_group = layer_group
                self.is_last = is_last

            def forward(self, *args, **kwargs):
                # assume first input to forward is hidden states
                if len(args) > 0:
                    hidden_states = args[0]
                    # del args
                else:
                    first_key = list(kwargs.keys())[0]
                    hidden_states = kwargs.pop(first_key)

                first_inputs[self.layer_group] = hidden_states

                # preserve rest of positional arguments
                layer_args[self.layer_group] = args[1:]
                layer_kwargs[self.layer_group] = kwargs
                
                # early exit for last group of layers
                if self.is_last:
                    raise ValueError

                return self.module.forward(*args, **kwargs)

        keys = list(layer_groups.keys())

        for i in range(len(keys)):
            layer_group = keys[i]
            is_last = True if i == len(keys) - 1 else False

            modules = layer_groups[layer_group]
            modules[0] = Catcher(modules[0], layer_group, is_last)

        try:
            self.run_model(calibration_set)
        except ValueError:
            pass
        
        for _, modules in layer_groups.items():
            # restore proper module at beginning of layer group
            modules[0] = modules[0].module
        
        return first_inputs, layer_args, layer_kwargs
       

    def _gather_linear_inputs(self, layer, named_linears, layer_group):

        def input_hook(module, input, output, module_name, inputs):
            x = input[0]
            x = x.detach().cpu()

            # out = output[0]
            # out = out.detach().cpu()

            inputs[module_name].append(x)
        

        inputs = defaultdict(list)
        hooks = []
        
        for name, mod in named_linears.items():
            hooks.append(
                mod.register_forward_hook(partial(input_hook,
                                                  module_name = name, 
                                                  inputs = inputs))
            )

        # compute next set of inputs, grabbing linear inputs through the hooks
        self.inps = layer(self.inps, *self.layer_args[layer_group], **self.layer_kwargs[layer_group])
        self.inps = self.inps[0]

        # remove hooks from model
        for hook in hooks:
            hook.remove()

        inputs = {k: torch.cat(v, dim=0) for k, v in inputs.items()}

        return inputs
    
    def _compute_scales(self, layer, prev_op, modules, inp, parent_module, layer_args, layer_kwargs):

        def clear_memory(weight=None):
            if weight is not None:
                del weight
            gc.collect()
            torch.cuda.empty_cache()
    
        inp = inp.to(device)

        # block of weights concatted together
        W = torch.cat([mod.weight for mod in modules], dim = 0)
        orig_shape = W.shape
        W = W.view(-1, self.group_size)

        # rescale W to 0-1 scale
        W_scale = W.abs() / (W.abs().amax(dim=1, keepdim=True) + 1e-6)
        W_scale = W_scale.view(orig_shape)
        # per channel mean of normalized weights
        W_mean = W_scale.mean(0)
        W_mean = W_mean.view(-1)

        clear_memory(W)

        # per channel mean of input (activation)
        X_mean = inp.abs().view(-1, inp.shape[-1]).mean(0)
        X_mean = X_mean.view(-1)

        kwargs = sanitize_kwargs(layer_kwargs, parent_module)

        # compute full precision output
        with torch.no_grad():
            # fp_output = parent_module(inp, *layer_args, **kwargs)[0]
            fp_output = parent_module(inp, **kwargs)[0]
            # print(fp_output)

        
        # Grid search for best scales
        n_grid = self.grid_search_size
        history = []
        best_ratio = -1
        best_scales = None
        best_error = float("inf")

        org_sd = {k: v.cpu() for k, v in parent_module.state_dict().items()}

        for ratio in range(n_grid):
            scales = X_mean.pow(ratio).clamp(min=1e-4).view(-1)

            # avoid scaling values that overflow
            scales[torch.isinf(scales)] = 1
            scales[torch.isnan(scales)] = 1

            scales_view = scales.view(1, -1).to(device)
            # print(scales_view)

            # Q(W * s)
            # NOTE: only nn.linear modules for now

            # pseudo-quantize modules
            for mod in modules:
                mod.weight.mul_(scales_view)
                mod.weight.data = (
                    self.pseudo_quantize_tensor(mod.weight.data)[0] / scales_view
                )

            with torch.no_grad():
                # Q(W * s) * X
                # q_output = parent_module(inp, *layer_args, **layer_kwargs)[0]
                # q_output = parent_module(inp, *layer_args, **layer_kwargs)[0]
                q_output = parent_module(inp, **kwargs)[0]
            
            # Compute loss (L2 NORM)
            loss = compute_loss(fp_output, q_output)

            history.append(loss)
            if loss < best_error:
                best_error = loss
                best_ratio = ratio
                best_scales = scales.clone()

            # reset to original weights
            parent_module.load_state_dict(org_sd)

        assert best_ratio != -1, "best scales ratio never set"
        assert torch.isnan(best_scales).sum() == 0, best_scales

        return best_scales.detach().cpu()
    

    def _apply_scales(self, scale, prev_op, modules):

        scale = scale.to(device)
        prev_op = prev_op.to(device)

        if isinstance(prev_op, torch.nn.LayerNorm):
            prev_op.weight.div_(scale)

            if hasattr(prev_op, "bias") and prev_op.bias is not None:
                prev_op.bias.div_(scale)

        elif isinstance(prev_op, torch.nn.Linear):
            prev_op.weight[-scale.size(0) :].div_(scale.view(-1, 1))

        for fc in modules:
            fc.weight.mul_(scale.view(1, -1))

        # SANITY checks
        for p in prev_op.parameters():
            assert torch.isnan(p).sum() == 0
        for fc in modules:
            for p in fc.parameters():
                assert torch.isnan(p).sum() == 0

        prev_op.cpu()
        for fc in modules:
            fc.cpu()
        scale.cpu()



    # return layers of model to consider for quantization (modify with config file)
    def _get_model_layer_groups(self):
        raise NotImplementedError('_get_model_layers')
    
    def _get_calibration_set(self):
        raise NotImplementedError('_get_calibration_set')

    def _prepare_input(self):
        raise NotImplementedError('_prepare_input')
    
    def _group_modules_for_scaling(self, layer, linear_inputs, layer_group):
        raise NotImplementedError('_group_modules_for_scaling')
    

class Blip2ForConditionalGenerationAWQQuantizer(BaseAWQQuantizer):

    def __init__(self, model, inputs_processor, dataset):
        assert isinstance(model, Blip2ForConditionalGeneration)

        super().__init__(model, device, inputs_processor, dataset)
        self.run_model = model.generate
        
    def _get_model_layer_groups(self):
        # NOTE: returning all layers for now
        # NOTE: should ensure that keys are defined sequentially for early quitting of calibration set run
        return {'vit_layers': self.model.vision_model.encoder.layers,
                'qformer_layers': self.model.qformer.encoder.layer,
                'llm_layers': self.model.language_model.model.decoder.layers
               }

    def _get_calibration_set(self):
        # NOTE: small set for testing for now

        samples = []
        # n = 0

        random.seed(0)
        indices = random.sample(range(len(self.dataset)), self.n_samples)
        
        for i in indices:
            
            data = self.dataset[i]

            sample = self._prepare_input(data[0])
            samples.append(sample)
            
            # n += 1
            # if n == self.n_samples:
            #     break
        
        # NOTE: might have to break this up into batches, check gpu size
        samples = torch.cat(samples, dim = 0)
        return samples

    def _prepare_input(self, inp):
        X = self.inputs_processor(images=inp, return_tensors="pt").to(device)
        return X['pixel_values']
    
    def _group_modules_for_scaling(self, layer, linear_inputs, layer_group):
        grouped_mods = []

        if layer_group == 'vit_layers':
            
            grouped_mods.append(
                # vit self-attn
                dict(
                    prev_op = layer.layer_norm1,
                    modules = [layer.self_attn.qkv],
                    inp = linear_inputs['self_attn.qkv'],
                    parent_module = layer.self_attn,
                    layer_args = self.layer_args[layer_group],
                    layer_kwargs = self.layer_kwargs[layer_group]
                )
            )

            grouped_mods.append(
                # vit fc1
                dict(
                    prev_op = layer.layer_norm2,
                    modules = [layer.mlp.fc1],
                    inp = linear_inputs['mlp.fc1'],
                    parent_module = layer.mlp.fc1,
                    layer_args = self.layer_args[layer_group],
                    layer_kwargs = self.layer_kwargs[layer_group]
                )
            )

            grouped_mods.append(
                 # vit fc2
                dict(
                    prev_op = layer.mlp.fc1,
                    modules = [layer.mlp.fc2],
                    inp = linear_inputs['mlp.fc2'],
                    parent_module = layer.mlp.fc2,
                    layer_args = self.layer_args[layer_group],
                    layer_kwargs = self.layer_kwargs[layer_group]
                )
            )

        elif layer_group == 'qformer_layers':
            # TODO:
            pass
        elif layer_group == 'llm_layers':
            
            assert layer.do_layer_norm_before, "llm do_layer_norm_before set to false"

            # llm attn
            grouped_mods.append(
                dict(
                    prev_op = layer.self_attn_layer_norm,
                    modules = [
                        layer.self_attn.q_proj,
                        layer.self_attn.k_proj,
                        layer.self_attn.v_proj,
                    ],
                    inp = linear_inputs['self_attn.q_proj'],
                    parent_module = layer.self_attn,
                    layer_args = self.layer_args[layer_group],
                    layer_kwargs = self.layer_kwargs[layer_group]
                )
            )

            # llm attn output
            grouped_mods.append(
                dict(
                    prev_op = layer.self_attn.v_proj,
                    modules = [layer.self_attn.out_proj],
                    inp = linear_inputs['self_attn.out_proj'],
                    parent_module = layer.self_attn.out_proj,
                    layer_args = self.layer_args[layer_group],
                    layer_kwargs = self.layer_kwargs[layer_group]

                )
            )

            # LLM FC1
            grouped_mods.append(
                dict(
                    prev_op = layer.final_layer_norm,
                    modules = [layer.fc1],
                    inp = linear_inputs['fc1'],
                    parent_module = layer.fc1,
                    layer_args = self.layer_args[layer_group],
                    layer_kwargs = self.layer_kwargs[layer_group]
                )
            )

          # LLM FC2
            grouped_mods.append(
                dict(
                    prev_op = layer.fc1,
                    modules = [layer.fc2],
                    inp = linear_inputs['fc2'],
                    parent_module = layer.fc2,
                    layer_args = self.layer_args[layer_group],
                    layer_kwargs = self.layer_kwargs[layer_group]
                )
            )

        return grouped_mods
        

class Blip2ForImageTextRetrievalAWQQuantizer(BaseAWQQuantizer):

    def __init__(self, model, device, inputs_processor, dataset):
        assert isinstance(model, Blip2ForImageTextRetrieval)
        super().__init__(model, device, inputs_processor, dataset)
        self.run_model = model.forward
        
    def _get_model_layer_groups(self):
        # NOTE: returning all layers for now
        return [*[layer for layer in self.model.vision_model.encoder.layers],
                *[layer for layer in self.model.qformer.encoder.layer]]

    def _get_calibration_set(self):
        return [self.dataset[0], self.dataset[1]]

    def _prepare_input(self, batch):
        X = self.processor(images=batch[0], text=batch[1][0], return_tensors="pt").to(device, torch.float16)
        return X


In [9]:
b = Blip2ForConditionalGenerationAWQQuantizer(model, processor, coco_dataset)


In [10]:
layers, first_inputs, layer_args, layer_kwargs, linear_inputs, scales = b.quantize()

Expanding inputs for image tokens in BLIP-2 should be done in processing. Please follow instruction here (https://gist.github.com/zucchini-nlp/e9f20b054fa322f84ac9311d9ab67042) to update your BLIP-2 model. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.
Quantizing vit_layers:   3%|▎         | 1/39 [00:00<00:27,  1.38it/s]

--------------------------------------------------------------------------------
Parameter containing:
tensor([ 4.6849e-05,  1.7414e-03,  3.2692e-03,  ..., -6.4087e-04,
        -6.9559e-05,  2.2221e-03], device='cuda:0', requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter containing:
tensor([ 4.6849e-05,  1.7414e-03,  3.2692e-03,  ..., -6.4087e-04,
        -6.9559e-05,  2.2221e-03], requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter containing:
tensor([-5.0068e-06, -3.0923e-04, -3.2330e-04,  ...,  1.9634e-04,
         1.8036e-02,  1.0341e-04], device='cuda:0', requires_grad=True)
--------------------------------------------------------------------------------
---------------------------------------------------

Quantizing vit_layers:   5%|▌         | 2/39 [00:01<00:24,  1.51it/s]

--------------------------------------------------------------------------------
Parameter containing:
tensor([1.5557e-05, 2.4988e-01, 3.2129e-01,  ..., 2.5757e-02, 2.1497e-01,
        1.3318e-01], device='cuda:0', requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter containing:
tensor([1.5557e-05, 2.4988e-01, 3.2129e-01,  ..., 2.5757e-02, 2.1497e-01,
        1.3318e-01], requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter containing:
tensor([0.0368, 0.1511, 0.1774,  ..., 0.0641, 0.1412, 0.1503], device='cuda:0',
       requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter con

Quantizing vit_layers:   8%|▊         | 3/39 [00:01<00:23,  1.56it/s]

...

--------------------------------------------------------------------------------
Parameter containing:
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0', requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter containing:
tensor([1., 1., 1.,  ..., 1., 1., 1.], requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter containing:
tensor([[-1.7029e-02,  3.5431e-02, -1.2955e-02,  ...,  3.2745e-02,
         -1.2711e-02,  3.5095e-02],
        [ 6.2744e-02,  2.1088e-02, -4.0009e-02,  ...,  6.5613e-02,
         -2.0615e-02,  8.0032e-03],
        [ 8.1062e-05, -4.0054e-03,  9.3079e-03,  ...,  6.7993e-02,
          2.8275e-02,  6.6605e-03],
        ...,
        [ 2.8595e-02,  2.3804e-02,  7.4997e-03,  ...,  1.6556e-02,
     

Quantizing llm_layers:  97%|█████████▋| 31/32 [00:40<00:01,  1.32s/it]

--------------------------------------------------------------------------------
Parameter containing:
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0', requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter containing:
tensor([0.8456, 1.7902, 1.4572,  ..., 1.4527, 1.2658, 1.2724],
       requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter containing:
tensor([[ 0.0218, -0.0257, -0.0276,  ..., -0.0017,  0.0001, -0.0077],
        [ 0.0186, -0.0139, -0.0032,  ...,  0.0062, -0.0143,  0.0398],
        [ 0.0229,  0.0123, -0.0166,  ..., -0.0050, -0.0067, -0.0262],
        ...,
        [ 0.0118, -0.0141, -0.0230,  ...,  0.0066,  0.0257,  0.0038],
        [ 0.0073, -0.0045, -0.0230,  ...,  0.0220,  0.0319,  0.0137],


Quantizing llm_layers: 100%|██████████| 32/32 [00:42<00:00,  1.32s/it]

--------------------------------------------------------------------------------
Parameter containing:
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0', requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter containing:
tensor([0.8034, 1.5570, 1.4686,  ..., 1.1771, 1.2233, 1.1835],
       requires_grad=True)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Parameter containing:
tensor([[-0.0473,  0.0355,  0.0174,  ..., -0.0111,  0.0204, -0.0035],
        [ 0.0108,  0.0119, -0.0045,  ...,  0.0150,  0.0060,  0.0087],
        [ 0.0720,  0.0006,  0.0039,  ...,  0.0004, -0.0162,  0.0208],
        ...,
        [ 0.0104, -0.0037,  0.0020,  ...,  0.0160, -0.0180,  0.0087],
        [ 0.0155, -0.0081,  0.0165,  ...,  0.0440, -0.0014,  0.0121],


In [15]:
scales

[]

In [30]:
layer_args['qformer_layers']

(tensor([[[[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
            -0., -0., -0., -0., -0., -0., -0., -0., -0.]]],
 
 
         [[[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
            -0., -0., -0., -0., -0., -0., -0., -0., -0.]]]], device='cuda:0'),
 None,
 tensor([[[-0.7125,  0.3542,  0.7332,  ..., -0.5284, -0.0729,  0.0879],
          [-0.9326, -0.9130, -0.3849,  ..., -0.3952, -0.8801,  0.9865],
          [-0.2113,  0.1189,  1.1975,  ...,  0.2811, -0.8168,  0.2782],
          ...,
          [ 0.2122, -0.7599, -0.0777,  ...,  0.5073,  0.3702, -0.0031],
          [ 0.6824, -0.3208,  0.6164,  ..., -0.0497,  1.5667,  0.4806],
          [ 0.7810, -0.7398,  1.0975,  ...,  0.1164, -1.0482, -0.7280]],
 
         [[ 0.0457,  0.5419,  0.7550,  ..., -0.0269, -0.6244,  0.9082],
          [-0.8845, -0.2183,  0.2585,  ...,  0.9610, -0.3391,  1.6280],
  

In [31]:
layer_kwargs['qformer_layers']

{}

In [24]:
layer_kwargs['vit_layers']

{'output_attentions': False}

In [15]:
print(linear_inputs.keys())

dict_keys(['self_attn.qkv', 'self_attn.projection', 'mlp.fc1', 'mlp.fc2'])


In [42]:
# model = Blip2ForImageTextRetrieval.from_pretrained("Salesforce/blip2-itm-vit-g", torch_dtype=torch.float16)
# processor = AutoProcessor.from_pretrained("Salesforce/blip2-itm-vit-g")
# model.to(device)

# device

In [13]:
layers['vit_layers'][0]

Blip2EncoderLayer(
  (self_attn): Blip2Attention(
    (dropout): Dropout(p=0.0, inplace=False)
    (qkv): Linear(in_features=1408, out_features=4224, bias=True)
    (projection): Linear(in_features=1408, out_features=1408, bias=True)
  )
  (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
  (mlp): Blip2MLP(
    (activation_fn): GELUActivation()
    (fc1): Linear(in_features=1408, out_features=6144, bias=True)
    (fc2): Linear(in_features=6144, out_features=1408, bias=True)
  )
  (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
)

In [37]:
linear_inputs['self_attn.qkv'].shape

torch.Size([2, 257, 1408])

In [20]:
first_inputs['vit_layers']

tensor([[[ 0.7319,  0.2039, -0.1177,  ...,  0.3483, -0.1315, -0.3481],
         [-0.2441,  0.8111, -0.0983,  ...,  0.0234, -0.1451, -0.7400],
         [-0.0418,  1.7882, -0.3203,  ..., -0.0219, -0.0488, -0.1417],
         ...,
         [-0.9605, -0.2865,  0.5448,  ..., -0.0762,  0.4271,  1.2226],
         [-0.6000, -0.1437,  0.1221,  ..., -0.1582,  0.1567,  1.4119],
         [-0.2725, -0.3828,  0.3872,  ...,  0.2089,  0.1482,  0.7765]],

        [[ 0.7319,  0.2039, -0.1177,  ...,  0.3483, -0.1315, -0.3481],
         [ 0.4066,  0.8089, -0.1578,  ...,  0.0691,  0.0124, -0.3917],
         [ 0.2490,  1.6557, -0.3386,  ..., -0.1113,  0.0690, -0.0610],
         ...,
         [-1.0813, -0.3526,  0.3304,  ..., -0.0324,  0.3346,  0.8685],
         [-0.5170, -0.1903, -0.0778,  ..., -0.1721,  0.1628,  1.2161],
         [-0.0057, -0.5496,  0.3215,  ...,  0.1134, -0.0510,  0.9223]]],
       device='cuda:0')

In [25]:
layer_args['vit_layers']

(None,)

In [23]:
layer_kwargs['vit_layers']

{'output_attentions': False}

In [28]:
out = layers['vit_layers'][0](first_inputs['vit_layers'], *layer_args['vit_layers'], **layer_kwargs['vit_layers'])

In [30]:
out[0]

tensor([[[ 0.3378, -0.1946, -0.0252,  ...,  0.6891, -0.1665, -0.2123],
         [-0.5796, -0.0850,  0.8516,  ...,  0.2505,  0.2570, -2.0609],
         [-0.1985,  1.0866,  0.2261,  ..., -0.4743, -0.8173, -0.4517],
         ...,
         [-1.9534, -0.5337,  0.5927,  ...,  0.0530,  0.0967,  1.0787],
         [-1.4451, -0.4682,  0.1352,  ...,  0.0619,  0.0195,  1.4128],
         [-0.4682, -1.1753,  0.9844,  ...,  0.6326,  1.3233,  0.8188]],

        [[ 0.0635, -0.3202,  0.1072,  ...,  0.6998, -0.0949, -0.1115],
         [-0.6654,  0.2025,  0.1473,  ...,  0.6025,  0.0049,  0.0392],
         [-0.9092,  1.0444, -0.1438,  ...,  0.4590,  0.1668,  0.3372],
         ...,
         [-1.1537, -1.1512,  1.0716,  ...,  0.6038, -0.2326,  0.4638],
         [-0.8844, -0.3487,  0.2901,  ...,  0.2318, -0.4852,  1.4794],
         [-0.9237, -0.8065,  0.7050,  ...,  0.6329,  0.0026,  1.1944]]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [31]:
layers['vit_layers'][1](out[0], *layer_args['vit_layers'],  **layer_kwargs['vit_layers'])

(tensor([[[ 0.0026, -0.0644, -0.1888,  ...,  1.0750,  0.1186,  0.5769],
          [-1.0819,  0.1315,  0.9809,  ...,  0.7254,  0.2461, -1.5226],
          [-0.2447,  0.9629,  0.4360,  ..., -0.0313, -1.1122, -0.2950],
          ...,
          [-2.0357,  0.1081,  0.3105,  ...,  0.5053,  0.0971,  1.0850],
          [-1.2883,  0.0584, -0.2668,  ...,  0.5149,  0.1054,  1.6861],
          [-0.4440, -1.2503,  0.4363,  ...,  1.1043,  0.8403,  0.6618]],
 
         [[-0.1361,  0.0898, -0.0243,  ...,  0.9252,  0.0271,  0.6004],
          [-0.6104,  0.3142, -0.1762,  ...,  0.7918, -0.2915,  0.5835],
          [-0.9518,  0.8436, -0.6670,  ...,  0.6580,  0.1580,  0.7574],
          ...,
          [-0.8174, -0.8920,  1.0149,  ...,  0.7384,  0.2734,  0.8379],
          [-1.1361, -0.0739, -0.0643,  ...,  0.7002,  0.1879,  2.2353],
          [-0.8765, -0.5891,  0.4028,  ...,  1.1088,  0.3058,  1.6114]]],
        device='cuda:0', grad_fn=<AddBackward0>),)

In [21]:
first_inputs['qformer_layers']

tensor([[[-0.7876, -0.3205, -0.0842,  ..., -0.6614, -0.0151, -0.5240],
         [-0.0952, -0.0247,  0.3759,  ..., -0.2078,  0.4916, -0.4537],
         [-0.5563,  0.5105, -0.6659,  ..., -0.2041,  0.5277,  0.8380],
         ...,
         [-0.1493,  1.2919,  1.5551,  ...,  0.2978, -1.4789,  0.2294],
         [ 0.2532,  0.0649, -0.7901,  ..., -0.4740, -1.6942, -0.6370],
         [ 0.3278, -0.4323,  0.2681,  ..., -0.4160,  0.3958, -0.1349]],

        [[-0.7876, -0.3205, -0.0842,  ..., -0.6614, -0.0151, -0.5240],
         [-0.0952, -0.0247,  0.3759,  ..., -0.2078,  0.4916, -0.4537],
         [-0.5563,  0.5105, -0.6659,  ..., -0.2041,  0.5277,  0.8380],
         ...,
         [-0.1493,  1.2919,  1.5551,  ...,  0.2978, -1.4789,  0.2294],
         [ 0.2532,  0.0649, -0.7901,  ..., -0.4740, -1.6942, -0.6370],
         [ 0.3278, -0.4323,  0.2681,  ..., -0.4160,  0.3958, -0.1349]]],
       device='cuda:0')

In [44]:
layers['qformer_layers'][0](*first_inputs['qformer_layers'], **layer_kwargs['qformer_layers'])

(tensor([[[-0.5611, -0.4313,  0.7348,  ..., -0.5339,  0.2271, -0.6419],
          [-0.0081,  2.1908,  0.5378,  ...,  0.0602,  0.0924, -0.4073],
          [-0.3419,  0.2889, -0.3378,  ..., -0.3128,  0.8518, -0.0093],
          ...,
          [-0.4123,  1.7034, -0.6390,  ...,  0.3584, -0.3071, -0.3509],
          [-0.3916,  0.2184,  0.2309,  ..., -0.4097,  0.2242, -0.1063],
          [ 0.1907,  1.5663,  0.5063,  ..., -0.2080,  0.2138, -0.3192]],
 
         [[-0.6695, -0.3799,  0.7295,  ..., -0.4814,  0.0236, -0.6724],
          [-0.1965, -0.3873, -0.5339,  ..., -0.6227,  0.0911,  0.0212],
          [-0.8092,  0.3860, -0.2228,  ..., -0.5321, -0.2148, -0.6384],
          ...,
          [-0.1741,  0.9254, -0.0700,  ...,  0.0874, -1.2012, -0.1439],
          [-0.3797,  0.2424, -0.4833,  ..., -0.7007, -0.8832, -0.7238],
          [ 0.4638, -0.7695, -0.3554,  ..., -0.5841, -0.0452, -0.0482]]],
        device='cuda:0', grad_fn=<NativeLayerNormBackward0>),
 (tensor([[[[-1.7426,  0.2527,  1.7284,

In [18]:
layer_kwargs['qformer_layers']

{}

In [22]:
first_inputs['llm_layers']

tensor([[[-1.3969e+00,  2.4953e-01, -5.2337e-01,  ..., -9.5912e-01,
           1.3994e+00, -1.5692e-02],
         [ 1.9842e+00, -1.8033e+00, -5.4722e-01,  ..., -4.5155e-01,
          -1.1304e+00, -2.2611e-01],
         [ 8.4684e-01, -1.9760e-01, -5.8955e-01,  ...,  7.3538e-01,
           2.6387e-02, -5.0607e-01],
         ...,
         [ 7.1303e-01,  1.2223e+00,  8.5825e-01,  ...,  4.4241e+00,
          -2.8862e-01,  1.0177e+00],
         [ 2.8392e+00, -1.2753e+00, -2.6297e-01,  ..., -6.6006e-02,
          -1.6678e+00, -8.7073e-01],
         [-3.2982e-02,  3.1891e-03, -3.8714e-03,  ..., -2.6846e-02,
           1.0907e-02,  8.5526e-03]],

        [[-6.8959e-01, -4.7141e-01, -3.1821e-01,  ...,  5.0050e-01,
           1.0790e+00, -1.5016e-01],
         [ 9.6880e-01, -1.3011e+00,  1.2982e-01,  ...,  2.1108e-01,
          -1.5340e+00, -1.8530e-01],
         [ 2.3157e+00,  1.2002e+00,  8.9333e-01,  ...,  8.5588e-01,
          -9.5109e-01,  1.1636e+00],
         ...,
         [ 1.1332e+00, -5

In [19]:
layer_kwargs['llm_layers']

{'attention_mask': tensor([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
 
 
         [[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]],
        device='cuda:0'),
 'layer_head_mask': None,
 'past_key_value': (tensor([[[[ 3.4119e+00, -1.4275e+00,  1.6319e+00,  ...,  5.2653e-01,
              2.4766e+00, -1.3366e+00],
            [ 1.7859e+00, -2.7154e-01,  8.6161e-01,  ..., -8.4367e-02,
              1.5484e+00,  4.6518e-01],
            [ 1.8167e+00, -5.6734e-01,  1.4872e+00,  ..., -7.3273e-01,
              6.8322e-01, -1.6523e-01],
            ...,
            [ 2.3114e+00, -3.1942e-01, -6.0153e-01,  ..., -1.4472e+00,
              1.0140e+00, -6.6369e-01],
            [ 6.2211e-01,  4.0976e-01, -

In [161]:
calib_set = b._get_calibration_set()
calib_set.shape

torch.Size([2, 3, 224, 224])

In [164]:

inps = []
layer_kwargs = {}
# get input and kwargs to layer 0
# with_kwargs is only supported in PyTorch 2.0
# use this Catcher hack for now
class Catcher(nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, *args, **kwargs):
        # assume first input to forward is hidden states
        if len(args) > 0:
            hidden_states = args[0]
            del args
        else:
            first_key = list(kwargs.keys())[0]
            hidden_states = kwargs.pop(first_key)

        inps.append(hidden_states)
        layer_kwargs.update(kwargs)
        raise ValueError  # early exit to break later inference


modules[0] = Catcher(modules[0])

try:
    model.generate(calib_set.to(next(model.parameters()).device))
except ValueError:
    pass


In [165]:
modules

ModuleList(
  (0): Catcher(
    (module): Catcher(
      (module): Catcher(
        (module): Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
  )
  (1-38): 38 x Blip2EncoderLayer(
    (self_attn): Blip2Attention(
      (dropout): Dropout(p=0.0, inplace=False)
      (qkv): Linear(in_features=1408, out_features=4224, bias=True)
      (projection): Linea

In [166]:
inps

[tensor([[[ 0.7319,  0.2039, -0.1177,  ...,  0.3483, -0.1315, -0.3481],
          [-0.2441,  0.8111, -0.0983,  ...,  0.0234, -0.1451, -0.7400],
          [-0.0418,  1.7882, -0.3203,  ..., -0.0219, -0.0488, -0.1417],
          ...,
          [-0.9605, -0.2865,  0.5448,  ..., -0.0762,  0.4271,  1.2226],
          [-0.6000, -0.1437,  0.1221,  ..., -0.1582,  0.1567,  1.4119],
          [-0.2725, -0.3828,  0.3872,  ...,  0.2089,  0.1482,  0.7765]],
 
         [[ 0.7319,  0.2039, -0.1177,  ...,  0.3483, -0.1315, -0.3481],
          [ 0.4066,  0.8089, -0.1578,  ...,  0.0691,  0.0124, -0.3917],
          [ 0.2490,  1.6557, -0.3386,  ..., -0.1113,  0.0690, -0.0610],
          ...,
          [-1.0813, -0.3526,  0.3304,  ..., -0.0324,  0.3346,  0.8685],
          [-0.5170, -0.1903, -0.0778,  ..., -0.1721,  0.1628,  1.2161],
          [-0.0057, -0.5496,  0.3215,  ...,  0.1134, -0.0510,  0.9223]]],
        device='cuda:0')]

In [ ]:
# b = Blip2ForImageTextRetrievalAWQQuantizer(model, processor, coco_dataset)
# inputs = b.quantize()


In [ ]:
# TODO: exlude certain linear layers, reading from quant config

In [ ]:
# TODO: solve for optimal (per input channel) scaling factor
# TODO: grid search for \alpha which balances protection of salient / non-salient weights